Import libraries and load in initial data for cleaning and restructuring

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import itertools

# Load the uploaded CSV files
file_paths = {
    "league_stats": "Call_Of_Duty_League_Stats.csv",
    "player_stats": "CDL_PlayerStats2025.csv",
    "roster": "CDL_Roster.csv",
    "team_stats": "CDL_Team_Stats.csv",
}


After merging the data and changing the name column to ID

In [1]:
# Re-import necessary libraries after reset
import pandas as pd

# Reload the uploaded dataset
file_path = 'Combined_Team_Data_with_Averages.csv'
data = pd.read_csv(file_path)

# Retain only the desired columns
columns_to_keep = ['ID', 'Current Team', 'Overall KD', 'Search Destroy KD', 'Control KD', 'Hardpoint KD']
cleaned_data = data[columns_to_keep]

# Save the cleaned dataset
cleaned_file_path = "Cleaned_Team_Data.csv"
cleaned_data.to_csv(cleaned_file_path, index=False)

cleaned_file_path.head()


'Cleaned_Team_Data.csv'

Create predictions for matchups based on stats of the overall team performance.

In [2]:
# Generate matchups and predict outcomes based on higher total stats
teams = cleaned_data['Current Team'].unique()
matchups = []

# Loop through all pairwise matchups
for i in range(len(teams)):
    for j in range(i + 1, len(teams)):
        team1 = teams[i]
        team2 = teams[j]
        
        # Get stats for both teams
        team1_stats = cleaned_data[cleaned_data['Current Team'] == team1][['Overall KD', 'Search Destroy KD', 'Control KD', 'Hardpoint KD']].sum().sum()
        team2_stats = cleaned_data[cleaned_data['Current Team'] == team2][['Overall KD', 'Search Destroy KD', 'Control KD', 'Hardpoint KD']].sum().sum()
        
        # Predict the winner
        winner = team1 if team1_stats > team2_stats else team2
        matchups.append({
            'Team A': team1,
            'Team B': team2,
            'Team A Total Stats': team1_stats,
            'Team B Total Stats': team2_stats,
            'Predicted Winner': winner
        })

# Convert the results into a DataFrame
matchups_df = pd.DataFrame(matchups)

# Display the predictions
print("Matchup Predictions:")
print(matchups_df.head())



Matchup Predictions:
      Team A      Team B  Team A Total Stats  Team B Total Stats  \
0  VAN Surge   Cloud9 NY              19.255            17.10000   
1  VAN Surge   TOR Ultra              19.255            21.44250   
2  VAN Surge  BOS Breach              19.255            20.80750   
3  VAN Surge    ATL FaZe              19.255            18.55875   
4  VAN Surge   OpTic TEX              19.255            20.92625   

  Predicted Winner  
0        VAN Surge  
1        TOR Ultra  
2       BOS Breach  
3        VAN Surge  
4        OpTic TEX  


In [3]:
# Features: Create differences in stats
matchups_df['Stat Diff'] = matchups_df['Team A Total Stats'] - matchups_df['Team B Total Stats']
matchups_df['Target'] = matchups_df['Predicted Winner'] == matchups_df['Team A']  # Team A wins = 1, otherwise 0
matchups_df['Target'] = matchups_df['Target'].astype(int)

# Features and target
X = matchups_df[['Stat Diff']]
y = matchups_df['Target']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Display results
print(f"Model Accuracy: {accuracy:.2%}")
print("Classification Report:")
print(report)

accuracy

NameError: name 'train_test_split' is not defined